In [1]:
%run "../../../common/0_notebooks_base_setup.py"
from dhtest import test_2_numpy_vectorizacion

/media/paulati/Nuevo vol/paula/dh/2021/dsad_2021/common
default checking
Running command `conda list`... ok
jupyterlab=2.2.6 already installed
pandas=1.1.5 already installed
bokeh=2.2.3 already installed
seaborn=0.11.0 already installed
matplotlib=3.3.2 already installed
ipywidgets=7.5.1 already installed
pytest=6.2.1 already installed
chardet=4.0.0 already installed
psutil=5.7.2 already installed
scipy=1.5.2 already installed
statsmodels=0.12.1 already installed
scikit-learn=0.23.2 already installed
xlrd=2.0.1 already installed
Running command `conda install --yes nltk=3.5.0`... ok
Solving environment: ...working... done

# All requested packages already installed.


unidecode=1.1.1 already installed
pydotplus=2.0.2 already installed
pandas-datareader=0.9.0 already installed
flask=1.1.2 already installed


---

<img src='../../../common/logo_DH.png' align='left' width=35%/>


# Numpy

<a id="section_toc"></a> 
## Tabla de Contenidos

[Intro](#section_intro)

[Universal Functions](#section_ufunc)

[Métodos matemáticos y estadísticos](#section_statistical)

[Broadcasting](#section_broadcasting)

---

## Vectorización

<a id="section_intro"></a> 
### Intro
[volver a TOC](#section_toc)

En esta notebook vamos a usar funciones que se ejecutan en forma vectorial, es decir que realizan operaciones sobre cada uno de los elementos de un array sin usar loops, y por eso son mucho más eficientes.

Las operaciones vectorizadas trabajan sobre los datos como un bloque.  Por eso es necesario que los tipos sean homogéneos entre todos los elementos. En operaciones vectorizadas no recorremos los elementos en orden. 

![Image](img/computacion_vectorial.jpg)

<a id="section_ufunc"></a> 
### Universal Functions 
[volver a TOC](#section_toc)

Son funciones que realizan operaciones sobre cada elemento de un array multidimensional.

Estas operaciones se paralelizan y el procesador las completa en un tiempo mucho menor que si aplicásemos un loop y operásemos sobre cada uno de los elementos.

A continuación tienen una lista de funciones ufunc unarias y binarias.

![Image](img/numpy_unary_ufunc.jpg)

![Image](img/numpy_binary_ufunc.jpg)

En el siguiente ejemplo vamos a comparar el tiempo de ejecución de calcular 1/x con un loop y con una ufunc.

`%timeit` nos devuelve el tiempo empleado en la ejecución de la función que recibe como argumento.

Construimos el array sobre el cual vamos a evaluar los distintos tiempos de ejecución

In [2]:
import numpy as np

# Definimos el tamaño del array que vamos a construir
big_array_size = 1000000

# Definimos la semilla del generador random
seed_cualquier_numero = 2843

# Creamos el generador 
random_generator_seed = np.random.default_rng(seed_cualquier_numero)

# Creamos el array con elementos de distribución uniforme
low = 1
high = 100
big_array = random_generator_seed.uniform(low, high, size=big_array_size)


In [3]:
# Enfoque tradicional usando un loop for para calcular 1/x de cada elemento del array
def compute_reciprocals(values):
    output = np.empty(len(values))
    for i in range(len(values)):
        output[i] = 1.0 / values[i]
    return output

# con -o podemos guardar el resultado de timeit en una variable, time_loop en este caso:
%timeit -o compute_reciprocals(big_array)

216 ms ± 6.41 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<TimeitResult : 216 ms ± 6.41 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)>

In [4]:
# esto asigna el valor del tiempo medido a una variable:
time_loop = _

In [5]:
time_loop_average = time_loop.average
time_loop_average

0.21614897114341147

In [6]:
# Enfoque broadcast (explicado mas adelante en esta notebook)
%timeit -o (1.0 / big_array)

587 µs ± 55.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


<TimeitResult : 587 µs ± 55.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)>

In [7]:
time_ufunc = _

In [8]:
time_ufunc_average = time_ufunc.average
time_ufunc_average

0.0005866341707140756

In [9]:
# Usando numpy.divide (ufunc) la performance es muy similar
%timeit -o np.divide(1.0, big_array)

696 µs ± 84.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


<TimeitResult : 696 µs ± 84.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)>

In [10]:
time_np_divide = _

In [11]:
time_np_divide_average = time_np_divide.average
time_np_divide_average

0.0006958353381425175

In [12]:
print("tiempo promedio loop: ", time_loop_average)
print("tiempo promedio ufunc: ", time_ufunc_average)
print("tiempo promedio np.divide: ", time_np_divide_average)
print("el loop emplea ", time_loop_average / time_ufunc_average, " veces más tiempo que ufunc")
print("el loop emplea ",time_loop_average / time_np_divide_average, " veces más tiempo que np.divide")

tiempo promedio loop:  0.21614897114341147
tiempo promedio ufunc:  0.0005866341707140756
tiempo promedio np.divide:  0.0006958353381425175
el loop emplea  368.4561553588089  veces más tiempo que ufunc
el loop emplea  310.6323569602051  veces más tiempo que np.divide


<a id="section_statistical"></a> 
### Métodos matemáticos y estadísticos
[volver a TOC](#section_toc)

La clase array implementa también métodos que calculan de forma eficiente estadísticas sobre un array.

Podemos hacer agregaciones como suma, promedio, desvío standard.

A continuación hay una lista de algunos de los métodos disponibles en Numpy:

![Image](img/numpy_array_statistical_methods.jpg)


Veamos algunos ejemplos de uso de estos métodos.

Construimos un array que usaremos para comparar tiempos de ejecución de distintos métodos:

In [13]:
# Definimos el tamaño del array que vamos a construir
big_array_size = 1000000

# Definimos la semilla del generador random
seed_cualquier_numero = 4703

# Creamos el generador 
random_generator_seed = np.random.default_rng(seed_cualquier_numero)

# Creamos el array
low = 1
high = 100
big_array = random_generator_seed.uniform(low, high, size=big_array_size)


Calculamos la suma de los elementos del array usando sum 
https://docs.python.org/3/library/functions.html#sum

Y medimos el tiempo empleado en esto

In [14]:
%timeit -o sum(big_array)

115 ms ± 2.67 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


<TimeitResult : 115 ms ± 2.67 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)>

In [15]:
python_sum_time = _

In [16]:
python_sum_time_average = python_sum_time.average
python_sum_time_average

0.11496314314286533

Calculamos ahora la suma de los elementos del array usando sum de Numpy
https://docs.scipy.org/doc/numpy/reference/generated/numpy.sum.html

Y medimos el tiempo empleado en esto

In [17]:
%timeit -o np.sum(big_array)

212 µs ± 27 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


<TimeitResult : 212 µs ± 27 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)>

In [18]:
numpy_sum_time = _

In [19]:
numpy_sum_time_average = numpy_sum_time.average
numpy_sum_time_average

0.00021239422071396672

In [20]:
print("Para este array la implementación sum de python emplea", python_sum_time_average / numpy_sum_time_average, 
      "veces más tiempo que la implementación sum de Numpy")

Para este array la implementación sum de python emplea 541.2724638006383 veces más tiempo que la implementación sum de Numpy


Ahora vamos a mostrar algunos ejemplos de estas funciones sobre una matriz (array de dos dimensiones).

In [21]:

# Definimos el tamaño de la matriz que vamos a construir
big_array_size = (300, 400)

# Definimos la semilla del generador random
seed_cualquier_numero = 4703

# Creamos el generador 
random_generator_seed = np.random.default_rng(seed_cualquier_numero)

# Creamos el array
low = 1
high = 100
big_array = random_generator_seed.uniform(low, high, size=big_array_size)


* Suma de todos los elementos de la matriz:

In [22]:
print("Suma de toda la matriz: ", big_array.sum())

Suma de toda la matriz:  6064843.1555379275


Es lo mismo que escribir:

In [23]:
np.sum(big_array)

6064843.1555379275

* Mínimo de cada columna, debe devolver un array de longitud igual a la cantidad de columnas de la matriz:

`axis` representa el eje que se va a **reducir**

En una matriz de dos dimensiones axis 0 representa el eje de las filas, y axis 1 el eje de las columnas

Para calcular los mínimos de cada columna debemos reducir las filas, por lo tanto el valor de axis es 0

In [24]:
minimos_por_columnas = big_array.min(axis=0)

cant_columnas = big_array.shape[1]

print("cantidad de columnas de big_array:", cant_columnas)

print("cantidad de elementos en los mínimos por columnas:",  len(minimos_por_columnas))

#print("Mínimos de cada columna: ", minimos_por_columnas)

cantidad de columnas de big_array: 400
cantidad de elementos en los mínimos por columnas: 400


* Máximo de cada fila, debe devolver un array de longitud igual a la cantidad de filas de la matriz:

`axis` representa el eje que se va a reducir, en este caso debemos reducir las columnas, por lo tanto el valor de axis es 1


In [25]:
maximos_por_filas = big_array.max(axis=1)

cant_filas = big_array.shape[0]

print("cantidad de filas de big_array:", cant_filas)

print("cantidad de elementos en los máximos por filas:",  len(maximos_por_filas))

#print("Máximos de cada fila: ", maximos_por_filas)


cantidad de filas de big_array: 300
cantidad de elementos en los máximos por filas: 300


* Suma de los elementos de cada fila

en este caso tenemos que reducir columnas, por lo tanto el valor de axis es 1

> Recuerden que `big_array.sum(axis=1)` es lo mismo que `np.sum(big_array, axis=1)`

In [26]:
#print("Suma de cada fila: ", big_array.sum(axis=1))
print("Suma de cada fila: ", np.sum(big_array, axis=1))

Suma de cada fila:  [18910.33036273 20279.14633071 20456.58297344 19796.15514547
 20312.11572238 20166.83973085 19571.65484108 20172.65906289
 20721.78910578 20739.00568135 19809.70790564 20612.61873513
 20425.50278738 19072.25495072 19568.41708099 19652.86151729
 20047.75205379 20147.09003941 19804.32615746 20350.81655519
 20273.33281219 18769.85722544 19006.4913321  20410.5636523
 20236.25697818 20117.99531226 19977.36466832 20933.91456867
 20418.5036841  20530.54326175 19540.2517708  21074.26635655
 20062.02056112 20900.32366252 19556.19555206 19466.89151054
 19189.42374796 20270.07301021 19798.0866719  21040.84115933
 20869.09701017 21220.02649443 20568.0745206  20335.83152359
 19391.49009386 19843.10897357 21137.342842   20348.00246468
 20784.49623609 20575.13714506 20708.73582908 20487.59057178
 20920.82869947 20368.86131967 20944.35872889 20794.20724457
 20820.72110523 21183.72676242 20784.95677375 19181.92092683
 19918.97842758 20160.39077051 20333.78465588 20592.33594627
 2076

<div id="caja5" style="float:left;width: 100%;">
  <div style="float:left;width: 15%;"><img src="../../../common/icons/ponete_a_prueba.png" style="align:left"/> </div>
  <div style="float:left;width: 85%;"><label>
<b>Ejercicios</b>
    
1) Generemos un array de 1 dimension con 1000 elementos con distribución normal de media 5 y desvío 2, inicialicemos la semilla en el valor 4703.
    
2) Usando algunas de las funciones de Numpy listadas en Métodos matemáticos y estadísticos, calculemos la media y el desvío de los elementos del array que construimos en el punto 1.
    
3) Generemos otro array de dimensiones 100 filas, 20 columnas con distribución normal de media 5 y desvío 2.
    
4) Usando las mismas funciones que en 2) calculemos la media y el desvío de cada fila del resultado de 3.
    
5) Usando las mismas funciones que en 2) calculemos la media y el desvío de cada columna del resultado de 3.
    
*¿Los resultados que obtuvieron son los que esperaban?*
    
**Ayudas**:
    
1) Generamos datos normales en 1_numpy.ipynb
    
4 y 5) Recuerden la existencia del parámetro `axis`

**Opcional**:
    
¿Pueden resolver los puntos 4) y 5) con la misma función?</label></div>
</div>



In [27]:
def ejercicio1_3(media=0, desvio=1, tupla_dim=(10)):
    # cambiar aqui:
    result = np.zeros(tupla_dim)
    return result

def ejercicio2_media(array):
    # cambiar aqui:
    result = -1
    return result

def ejercicio2_desvio(array):
    # cambiar aqui:
    result = -1
    return result

def ejercicio4_media_filas(data):
    # cambiar aqui:
    result = np.ones(10)
    return result

def ejercicio4_desvio_filas(data):
    # cambiar aqui:
    result = np.ones(10)
    return result

def ejercicio5_media_columnas(data):
    # cambiar aqui:
    result = np.ones(10)
    return result

def ejercicio5_desvio_columnas(data):
    # cambiar aqui:
    result = np.ones(10)
    return result

import numpy as np

########################################
# completar con los parametros adecuados
########################################
datos = ejercicio1_3()
print("ej 1:", test_2_numpy_vectorizacion.test_ejercicio1(datos))

result2_media = ejercicio2_media(datos)
print("ej 2 media:", test_2_numpy_vectorizacion.test_ejercicio2_media(datos, result2_media))

result2_desvio = ejercicio2_desvio(datos)
print("ej 2 desvio:", test_2_numpy_vectorizacion.test_ejercicio2_desvio(datos, result2_desvio))

########################################
# completar con los parametros adecuados
########################################
datos_matrix = ejercicio1_3(tupla_dim=(10,10))
print("ej 3:", test_2_numpy_vectorizacion.test_ejercicio3(datos_matrix))

result4_media_filas = ejercicio4_media_filas(datos_matrix)
print("ej 4 media filas:", test_2_numpy_vectorizacion.test_ejercicio4_media_filas(datos_matrix, result4_media_filas))

result4_desvio_filas = ejercicio4_desvio_filas(datos_matrix)
print("ej 4 desvio filas:", test_2_numpy_vectorizacion.test_ejercicio4_desvio_filas(datos_matrix, result4_desvio_filas))

result5_media_columnas = ejercicio5_media_columnas(datos_matrix)
print("ej 5 media columnas:", test_2_numpy_vectorizacion.test_ejercicio5_media_columnas(datos_matrix, result5_media_columnas))

result5_desvio_columnas = ejercicio5_desvio_columnas(datos_matrix)
print("ej 5 desvio columnas:", test_2_numpy_vectorizacion.test_ejercicio5_desvio_columnas(datos_matrix, result5_desvio_columnas))


ej 1: Los valores generados no tienen la forma (tamaño) pedida
ej 2 media: result no es la media de los datos pasados como parámetro
ej 2 desvio: result no es el desvío de los datos pasados como parámetro
ej 3: Los valores generados no tienen la forma (tamaño) pedida
ej 4 media filas: result no es la media de los datos pasados como parámetro
ej 4 desvio filas: result no es el desvío de los datos pasados como parámetro
ej 5 media columnas: result no es la media de los datos pasados como parámetro
ej 5 desvio columnas: result no es el desvío de los datos pasados como parámetro


<a id="section_broadcasting"></a> 
### Broadcasting
[volver a TOC](#section_toc)

En conjunto con las ufuncs, el broadcasting es una forma de aplicar operaciones sobre los datos sin tener que escribir loops "for" en Python nativo que resultan más lentos.

Recordemos que cuando operamos sobre arrays de las mismas dimensiones, se pueden hacer operaciones eficientes elemento a elemento.

Numpy tiene un conjunto de reglas para aplicar operaciones elemento a elemento en arrays de diferente tamaño. 
Se proyectan los valores de los arrays igualando las dimensiones de los argumentos, para poder operar sobre los mismos de forma vecotrizada.

![Image](img/broadcasting.jpg)

El broadcasting en NumPy sigue un conjunto estricto de reglas para determinar la interacción entre las dos arrays:

* Regla 1: si los dos arrays difieren en su número de dimensiones (forma), se rellena con 1s a su izquierda aquel que tiene menos dimensiones. 

* Regla 2: si el tamaño de los dos arrays no coincide en alguna dimensión, el array con tamaño igual a 1 en esa dimensión se estira para que coincida con el tamaño del otro.

* Regla 3: si en alguna dimensión los tamaños son diferentes y ninguno es igual a 1, se genera un error.

In [28]:
a = np.array([0, 1, 2])
b = np.array([5, 5, 5])
a + b

array([5, 6, 7])

En el ejemplo de arriba, "a + b" es una operación eficiente porque "a" y "b" tienen la mismas dimensiones. 

Las reglas de "broadcasting" de Numpy, permiten que la operación siga siendo eficiente llevando los elementos involucrados a la misma dimensión y tamaño.


Veamos en código los ejemplos de la imagen:

In [29]:
# linea 1 de la imagen:

a = np.array([0, 1, 2])
b = 5
a + b

array([5, 6, 7])

In [30]:
# linea 2 de la imagen:
a = np.ones((3, 3))
b = np.array([0, 1, 2])
a + b

array([[1., 2., 3.],
       [1., 2., 3.],
       [1., 2., 3.]])

In [31]:
# linea 3 de la imagen:

In [32]:
a = np.array([[0], [1], [2]])
print(a.shape)
a

(3, 1)


array([[0],
       [1],
       [2]])

In [33]:
b = np.array([0, 1, 2])
print(b.shape)
b

(3,)


array([0, 1, 2])

In [34]:
a + b

array([[0, 1, 2],
       [1, 2, 3],
       [2, 3, 4]])